# Лабораторная работа: Капрекар и неподвижные точки функции К(i)
* * * * * * * * * * * *
### Вычитание в соответствующей системе счисления
На вход поступает два массива list и основание системы счисления, происходит поразрядное вычитание

In [1]:
bad = ['A', 'B', 'C', 'D', 'E', 'F']

def substr(a, b, num_sys):
    res = []
    for i in range(len(b)):
        if (b[i] - a[i] >= 0): res.append(b[i] - a[i])
        else: 
            res.append(b[i] + num_sys - a[i])
            k = i + 1
            while ((b[k] == 0) and (k < len(b))):
                b[k] = b[k] + num_sys - 1
                k = k + 1
            #if (k == len(b):  - число < 0, в рамках наших вычислений такого не случится... можно добавить исключение
            b[k] = b[k] - 1
    ## для единственности найденных чисел сразу отсортируем его по убыванию
    res.reverse()
    for i in range(len(res)): 
        if(res[i] > 9): res[i] = bad[res[i] - 10]
    return ("".join(map(str,res)))

### Функция Капрекар. 
На вход поступает строка, в которой записано число, количество разрядов числа и система счисления  
На выходе функция возвращает строку - результат вычислений

In [2]:
def kaprekar(number, digit, num_sys):
    list_digits = [int(d, num_sys) for d in number]
    #while(digit > len(elem)):
    #    digits.append('0')
    list_digits.sort(reverse=True)
    max_sq = list_digits.copy()
    list_digits.sort()
    min_sq = list_digits.copy()
    #min_num = int("".join(map(str,min_sq))
    #max_num = int("".join(map(str,max_sq)))
    return substr(max_sq, min_sq, num_sys)

#### Введем переменные:  
list magicNumber содержит информацию о всех найденных неподвижных точках.  
            [ строка-запись числа - ключ, система счисления и длина до магического числа - пара значений по ключу]  
  
list usedNumber вспомогательный, чтобы найти цикл.

In [3]:
import math
magicNumber = {0: (math.inf, 2)}
usedNumber = []
Cycles = []

### Построение всех различных (для функции) чисел

Как будем строить число? Начнем с малого, заполним набор всех чисел для случая 2-разрядного числа с системе счисления по основанию 2. Это числа "11" и "10". Как далее происходит переход к большей разрядности или основанию С.С.?  

#### Разрядность:  
11 10       ->  (111, 110) + (100)  
111 110 100 -> (1111, 1110) + (1100) + (1000)   

#### Система счисления:  
11 10       + 22 21 20  
111 110 100 + 222 221 220 211 210 200


In [4]:
import numpy as np
batch_size = 10
# начальный массив возможных значений для d = 2, r = 2
start = ["11", "10"]

# строим таким образом, чтобы были только max значения (можно все min или премешанноые в виде 51423, 
# главное, чтобы одному набору цифр соответствовала только одна комбинация из них)

def add_d(arr, digit):
    tmp = []
    for i in arr:
        for k in range(3):
            if (int(i[digit - 2]) < k): break
            else: tmp.append(i + str(k))
    return tmp

def add_r(rez, number, digit, r):
    if (r > 10): number = number + bad[r - 11]
    else: number = number + (str(r - 1))
    for i in range (r):
        if (digit <= 1):
            rez.append(number)
            return rez
        else:
            add_r(rez, number, digit - 1, r - i)
    return rez

def in_Cycles(r):
    for arr in Cycles:
        if (arr[1] == r):
            for number in arr[0]:
                if (usedNumber[0] == number):
                    return True
    return False

### Process функция
На вход получает число, применяет к нему функцию капрекара, пока не найдется магическое число или процесс не зациклится

In [5]:
def process(number, d, r):
    usedNumber.clear()
    count = 0
    prev = "0"
    while True:
        prev = number
        usedNumber.append(number)
        number = kaprekar(number, d, r) 
        count = count + 1 
        if (int(number,r) == 0):
            break
        #else: 
            #print ("number is " + number)
        if (number == prev):
            if (magicNumber.get(number) == None):
                magicNumber[number] = (r, count)
                #print ("число " + number + " является неподвижной точкой: d = " + str(d) +" r = " + str(r) + "\n")
            elif (magicNumber[number][1] < count):
                magicNumber[number] = (r, count)
            break
        else:
            if(number in usedNumber): #найден цикл
                while (usedNumber[0] != number):
                    del usedNumber[0]
                if (not in_Cycles(r)): Cycles.append((usedNumber[:], r, count))
                break

In [6]:
for d in range(2,17):
    if (d > 2): start = add_d(start, d)
    for r in range(2,17):
        if (r > 2): 
            collection.extend(add_r([],"", d, r))
        else: collection = start.copy()
        #print ("d = ",d,"r = ",r)
        # 19 раз берем рандомные числа из collection, если их больше, чем batch_size, и находим магические числа или циклы
        # можно установить число побольше и подождать подольше =) 
        for n in range (20):
            if (len(collection) > batch_size):
                order = np.random.permutation(len(collection))[0 : min(d * 5, batch_size)]
                x = np.array(collection)[order]
            else:
                x = np.array(collection)
                
            for number in x:
                process(number, d, r)
            if (len(x) < batch_size):
                break

KeyboardInterrupt: 

In [7]:
magicNumber

{0: (inf, 2),
 '01': (2, 2),
 '13': (5, 4),
 '25': (8, 3),
 '37': (11, 5),
 '49': (14, 3),
 '011': (2, 2),
 '132': (4, 4),
 '253': (6, 5),
 '374': (8, 6),
 '495': (10, 7),
 '5B6': (12, 8),
 '6D7': (14, 9),
 '7F8': (16, 10),
 '1001': (2, 2),
 '0111': (2, 2),
 '3021': (4, 4),
 '3032': (5, 5),
 '6174': (10, 8),
 '92B6': (15, 3),
 '10101': (2, 2),
 '01111': (2, 2),
 '20211': (3, 5),
 '41532': (6, 7),
 '62853': (9, 3),
 '83B74': (12, 13),
 'A4E95': (15, 14),
 '101101': (2, 2),
 '110001': (2, 2),
 '011111': (2, 2),
 '310221': (4, 7),
 '330201': (4, 3),
 '213312': (4, 2),
 '420432': (6, 6),
 '325523': (6, 2),
 '530421': (6, 4),
 '640632': (8, 16),
 '631764': (10, 5),
 '65BB56': (12, 2),
 '951A74': (13, 21),
 '76DD67': (14, 2),
 '1011101': (2, 2),
 '1101001': (2, 2),
 '0111111': (2, 2),
 '2202101': (3, 3),
 '3203211': (4, 7),
 '6417532': (8, 5),
 '962B853': (12, 5),
 'C83FB74': (16, 9),
 '10111101': (2, 2),
 '11011001': (2, 2),
 '11100001': (2, 2),
 '01111111': (2, 2),
 '21022111': (3, 6),
 '3

In [8]:
Cycles

[(['03', '21'], 4, 3),
 (['23', '05', '41'], 6, 5),
 (['07', '61', '43'], 8, 4),
 (['53', '17'], 9, 5),
 (['09', '81', '63', '27', '45'], 10, 7),
 (['29', '65', '0B', 'A1', '83', '47'], 12, 7),
 (['1B', '93', '57'], 13, 6),
 (['2B', '85'], 14, 4),
 (['A3', '67', '0D', 'C1'], 14, 6),
 (['0F', 'E1', 'C3', '87'], 16, 5),
 (['4B', '69', '2D', 'A5'], 16, 6),
 (['022', '121'], 3, 3),
 (['242', '143'], 5, 5),
 (['264', '363'], 7, 3),
 (['484', '385'], 9, 4),
 (['5A5', '4A6'], 11, 6),
 (['6C6', '5C7'], 13, 9),
 (['7E7', '6E8'], 15, 10),
 (['1221', '1012'], 3, 3),
 (['1332', '2022'], 4, 4),
 (['3552', '3133', '1554', '4042', '4132', '3043'], 6, 8),
 (['5052', '5232', '3054'], 7, 5),
 (['3774', '4244', '1776', '6062', '6332'], 8, 10),
 (['6152', '5243', '3065'], 8, 6),
 (['5254', '3076', '7252'], 9, 6),
 (['7072', '7432', '5074'], 9, 7),
 (['7274', '5276', '5096', '9092', '9632'], 11, 7),
 (['7094', '9272', '7454', '3098', '9452'], 11, 7),
 (['8374', '5287', '6196', '7BB4', '7375', '4198'], 12, 